In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import talib as ta

from sklearn import svm
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [9]:
df = pd.read_csv("MCX.csv")
df = df.drop(df.columns[[0]], axis=1)
df[['Uopen', 'Uclose', 'Ulow', 'Uhigh']] = df[['Uopen', 'Uclose', 'Ulow', 'Uhigh']].apply(pd.to_numeric)
df.head()
df.shape

(2464, 4)

In [10]:
N = 2
w = 2*N + 1

for i in range(0,df.shape[0]-1):
    df.loc[df.index[i],'c'] = df['Uclose'][i] - df['Uclose'][i+1]
    
df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c
0,3828,3801,3888,3820,-7.0
1,3854,3818,3919,3827,-66.0
2,3858,3802,3904,3893,54.0
3,3822,3793,3869,3839,4.0
4,3859,3769,3865,3835,-59.0
5,3885,3862,3907,3894,-16.0
6,3919,3889,3942,3910,38.0
7,3823,3817,3890,3872,59.0
8,3819,3801,3852,3813,-12.0
9,3911,3794,3918,3825,-74.0


In [4]:
for i in range(0,df.shape[0]-(2*N+1)):
    s = 0
    for j in range((2*N)+1):
        s += df['Uclose'][i+j]
    s = s*1.0
    s = s/((2*N)+1)
    df.loc[df.index[i+N],'ma'] = s

df.head()

,Uopen,Ulow,Uhigh,Uclose,c,ma
0,3828,3801,3888,3820,-7.0,NaN
1,3854,3818,3919,3827,-66.0,NaN
2,3858,3802,3904,3893,54.0,3842.8
3,3822,3793,3869,3839,4.0,3857.6
4,3859,3769,3865,3835,-59.0,3874.2


In [5]:
for i in range(0,df.shape[0]-N):
    h = df['Uhigh'][i]
    l = df['Ulow'][i]
    for j in range(N):
        if h<df['Uhigh'][i+j]:
            h = df['Uhigh'][i+j]
        if l>df['Ulow'][i+j]:
            l = df['Ulow'][i+j]
    R = (h - df['Uclose'][i+N])*(-100)/(h-l)
    df.loc[df.index[i+N],'R'] = R
    

df.head()

/home/prakriti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in long_scalars
  if __name__ == '__main__':


,Uopen,Ulow,Uhigh,Uclose,c,ma,R
0,3828,3801,3888,3820,-7.0,NaN,NaN
1,3854,3818,3919,3827,-66.0,NaN,NaN
2,3858,3802,3904,3893,54.0,3842.8,-22.033898
3,3822,3793,3869,3839,4.0,3857.6,-68.376068
4,3859,3769,3865,3835,-59.0,3874.2,-62.162162


In [6]:
for i in range(0,df.shape[0]-N):
    h = 0
    l = 0
    vr = 0
    for j in range(N):
        h += df['Uhigh'][i+j]
        l += df['Ulow'][i+j]
    h = h*1.0/N
    l = l*1.0/N
    x = (df['Uhigh'][i+N] + df['Uclose'][i+N-1])/2
    y = (df['Ulow'][i+N] + df['Uclose'][i+N-1])/2
    if x!=y:
        vr = (h-l)*1.0/(x-y)
    df.loc[df.index[i+N],'VR'] = vr
    

df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR
0,3828,3801,3888,3820,-7.0,NaN,NaN,NaN
1,3854,3818,3919,3827,-66.0,NaN,NaN,NaN
2,3858,3802,3904,3893,54.0,3842.8,-22.033898,1.843137
3,3822,3793,3869,3839,4.0,3857.6,-68.376068,2.671053
4,3859,3769,3865,3835,-59.0,3874.2,-62.162162,1.854167
5,3885,3862,3907,3894,-16.0,3870.0,25.000000,3.822222
6,3919,3889,3942,3910,38.0,3864.8,2.173913,2.660377
7,3823,3817,3890,3872,59.0,3862.8,-87.500000,1.342466
8,3819,3801,3852,3813,-12.0,3863.8,-103.200000,2.470588
9,3911,3794,3918,3825,-74.0,3856.0,-73.033708,1.000000


In [7]:
df['RSI'] = ta.RSI(df['Uclose'].values, timeperiod = N)

df.head(10)

Exception: input array type is not double

In [ ]:
df['EMA'] = df.iloc[:,3].ewm(span=N,adjust=False).mean()

df.head()

In [ ]:
M = pd.Series(df['Uclose'].diff(1))  
aM = abs(M) 
r = N+1
s = N+1
EMA1 = M.ewm(span = r, min_periods = r - 1).mean()  
aEMA1 = aM.ewm(span = r, min_periods = r - 1).mean()
EMA2 = EMA1.ewm(span = s, min_periods = s - 1).mean() 
aEMA2 = aEMA1.ewm(span = s, min_periods = s - 1).mean()
TSI = pd.Series(EMA2 / aEMA2, name = 'TSI_' + str(r) + '_' + str(s))  
df['TSI'] = TSI

df.head(10)

In [ ]:
for i in range(0,df.shape[0]-N-1):
    bs = ''
    for j in range(N+1):
        if df['c'][i+j]>=0:
            bs+= '0'
        else:
            bs+= '1' 
    bs = int(bs, 2)
    df.loc[df.index[i+N],'y'] = bs

df.head(10)

In [ ]:
df = df.iloc[3:]
df = df.iloc[:2598]


In [ ]:
X = df.iloc[:, 5:-1]
y = df.iloc[:, -1]
split = 2200
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

clf = svm.SVC(kernel = 'poly')
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print('accuracy:',confidence)
predictions = clf.predict(X_test)
print('predicted class counts:',Counter(predictions))

# confidence = 0
# clf = svm.SVC(kernel = 'poly')
# for i in range(2594):
#     split = i+3
#     X_train, X_test, y_train, y_test = X[i:split], X[split:split+1], y[i:split], y[split:split+1]
#     clf.fit(X_train, y_train)
#     confidence += clf.score(X_test, y_test)
#     predictions = clf.predict(X_test)
    
# print(confidence/2594)